In [3]:
# import des librairies
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math

# srapping
import requests 
from bs4 import BeautifulSoup as bs
import re

# clean data
import ast
import dateparser
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

#export data
import time
import os
from datetime import date
from datetime import datetime

In [5]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/selmane/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/selmane/nltk_data...


True

In [6]:
# fonctions
def scrapping():
    job_offers = []
    page = 0
    # Select URL
    url = f'https://candidat.pole-emploi.fr/offres/recherche?emission=7&motsCles=data&offresPartenaires=true&range=0-19&rayon=10&tri=0'
    while True:
        
        response = requests.get(url)
        
#         # Check if the URL is not recognized or does not exist
#         if response.status_code != 200:
#             break
            
        html_content = response.text
            
        with open ('page.html', 'w', encoding='utf-8') as file:
            file.write(html_content)

        with open('page.html', 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = bs(html_content, 'html.parser')

        job_elements = soup.find_all('div', class_='media-body')
        
        for job_element in job_elements:
            # Extract the data-intitule-offre value from the job summary element
            
            if job_element.find('h2') and job_element.find('h2')['data-intitule-offre']:
                data_intitule_offre = job_element.find('h2')['data-intitule-offre']
            
                # Construct the detail page URL using the data-intitule-offre value
                detail_page_url = f'https://candidat.pole-emploi.fr/offres/recherche/detail/{data_intitule_offre}'

                # Scrape job details from the detail page
                detail_page_response = requests.get(detail_page_url)
                detail_page_content = detail_page_response.text

                with open ('page.html', 'w', encoding='utf-8') as file:
                    file.write(detail_page_content)

                with open('page.html', 'r', encoding='utf-8') as file:
                    detail_page_content = file.read()

                detail_page_soup = bs(detail_page_content, 'html.parser')
                
                # Liste des variables à extraire
                variables = {
                    'jobs': ('span', 'title'),
                    'description': ('div', 'description col-sm-8 col-md-7'),
                    'loc': ('span', 'name'),
                    'datePosted': ('span', 'datePosted'),
                    'workhours': ('dd', 'workHours'),
                    'salary': ('span', 'baseSalary'),
                    'contract_type': ('dl', 'icon-group'),
                    'experience': ('span', 'experienceRequirements'),
                    'qualifications': ('span', 'qualifications'),
                    'industry': ('span', 'industry'),
                    'company': ('h3', 't4 title'),
                    'size': ('div', 'media-body'),
                    'company_description': ('p', 'italic'),
                    'diploma': ('span', 'educationRequirements')
                }

                # Boucle pour extraire les variables
                for var, (tag, class_) in variables.items():
                    try:
                        if var in ['salary', 'contract_type']:
                            if var == 'salary':
                                try:
                                    salary_raw = detail_page_soup.find(tag, itemprop=class_)
                                    min_value = salary_raw.find('span', itemprop='minValue').get('content').strip()
                                    max_value = salary_raw.find('span', itemprop='maxValue').get('content').strip()
                                    globals()[var] = f"{min_value} - {max_value}"
                                except AttributeError:
                                    salary_match = re.search(r'content="([\d.]+)" itemprop="value"', str(salary_raw))
                                    globals()[var] = salary_match.group(1) if salary_match else math.nan
                                    
                            elif var == 'contract_type':
                                contract_type_raw = detail_page_soup.find(tag, class_=class_).find('dd')
                                globals()[var] = str(contract_type_raw.contents[0]).strip()
                                
                        else:
                            globals()[var] = detail_page_soup.find(tag, itemprop=class_).text.replace('\n', '').strip()
                    except AttributeError:
                        # En cas d'erreur
                        globals()[var] = math.nan


                # Extraction des autres variables spécifiques
                try:
                    skills_raw = detail_page_soup.find_all('span', itemprop='skills')
                    skills = [skill.text.strip() for skill in skills_raw]
                except AttributeError:
                    # En cas d'erreur
                    skills_raw = ''
                    skills = math.nan
                    
                try:
                    company = detail_page_soup.find('h3', class_='t4 title').text.strip()
                except AttributeError:
                    company = math.nan
                    
                try:
                    size = detail_page_soup.find('div', class_='media-body').find('p').text.strip()
                except AttributeError:
                    size = math.nan
                    
                try:                    
                    company_description = detail_page_soup.find('p', class_='italic').text.replace('\n', '').strip()
                except AttributeError:
                    company_description = math.nan
                
                try: 
                    description = detail_page_soup.find('div', class_='description col-sm-8 col-md-7').text.strip()
                except AttributeError:
                    description = math.nan
                    

                # Create a dictionary for the scraped data
                data = {
                    'jobs': jobs,
                    'description': description,
                    'loc': loc,
                    'datePosted': datePosted,
                    'workhours': workhours,
                    'salary': salary,
                    'contract_type': contract_type,
                    'experience': experience,
                    'diploma': diploma,
                    'skills': skills,
                    'qualifications': qualifications,
                    'industry': industry,
                    'company': company,
                    'size': size,
                    'company_description': company_description
                }

                job_offers.append(data)
            
            else : 
                print(f'fin de la page {page}!')

        # Extract the href of the next page button
        next_page_link = soup.find('div', id='zoneAfficherPlus').find('a')
        if next_page_link:
            href_main_page = next_page_link['href']
        else:
            break
        
        # Construct the URL for the next page
        next_page_url = f'https://candidat.pole-emploi.fr{href_main_page}'
        
        # Delay before navigating to the next page
        time.sleep(60)
        
        # Update the URL to the next page
        url = next_page_url
        page += 1

    # Transform the dictionary into a dataframe
    df = pd.DataFrame(job_offers)
    
    return df

In [7]:
df = scrapping()

fin de la page 0!
fin de la page 1!
fin de la page 2!
fin de la page 3!
fin de la page 4!
fin de la page 5!
fin de la page 6!
fin de la page 7!
fin de la page 8!
fin de la page 9!
fin de la page 10!
fin de la page 11!
fin de la page 12!
fin de la page 13!
fin de la page 14!
fin de la page 15!
fin de la page 16!
fin de la page 17!
fin de la page 18!
fin de la page 19!
fin de la page 20!
fin de la page 21!
fin de la page 22!
fin de la page 23!
fin de la page 24!
fin de la page 25!
fin de la page 26!
fin de la page 27!
fin de la page 28!
fin de la page 29!
fin de la page 30!
fin de la page 31!
fin de la page 32!


In [8]:
display(df.head(3))
print(df.shape)

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description
0,"RESPONSABL SUPPLYCHAIN, LOGISTIQUE, DATA (H/F)",Le Groupe BIOTECH DENTAL renforce ses équipes ...,74 - SCIONZIER,Actualisé le 18 août 2023,39H Travail en journée,42000.0,Contrat à durée indéterminée,4 ans,"Bac+3, Bac+4 ou équivalents - Gestion producti...",[Analyser les coûts de la chaîne logistique (s...,Cadre,Décolletage,BIOTECH DENTAL MANUFACTURING,50 à 99 salariés,Fabricant de composants et sous ensembles méca...
1,Biostatisticien/Data-Manager (H/F),Avec plus de 20 ans d'expérience dans le dével...,75 - PARIS 15,Actualisé le 18 août 2023,39H Travail en journée,28000.0 - 30000.0,Contrat à durée déterminée - 12 Mois,Débutant accepté,NaN,"[Analyse statistique, Analyser, exploiter, str...",Agent de maîtrise,Recherche-développement en autres sciences phy...,Association ARTIC,6 à 9 salariés,Avec plus de 20 ans d expérience dans le dével...
2,Data Analyst (H/F),Vous intégrez notre équipe Data pour interveni...,35 - RENNES,Actualisé le 18 août 2023,37H Travail en journée,35000.0 - 37000.0,Contrat à durée indéterminée,2 ans,Bac+5 et plus ou équivalents informatique déci...,"[Analyse statistique, Analyser, exploiter, str...",Cadre,Conseil en systèmes et logiciels informatiques,INETUM,0 salarié (n'ayant pas d'effectif au 31/12 mai...,NaN


(660, 15)


In [9]:
df.shape

(660, 15)

### Export df

In [10]:
# def export_raw_data(df):
#     csv_file_path = f"../data/bronze/raw_data.csv"
#     df.to_csv(csv_file_path, index=False)

In [11]:
# export_raw_data(df)

## Transform data

In [12]:
tools_list = ["AWS S3", "Azure Blob Storage", "Google Cloud Storage", "Dropbox", "Box", "OneDrive", "Salesforce", "HubSpot",
               "Slack", "Microsoft Teams", "Google Workspace", "Trello", "Jira", "Confluence", "GitHub", "GitLab", "Jenkins",
               "CircleCI", "Travis CI", "Docker", "Kubernetes", "Apache Hadoop", "Apache Spark", "Apache Kafka", "Apache Storm",
               "Elasticsearch", "Logstash", "Kibana", "Grafana", "Prometheus", "Nagios", "Splunk", "Tableau", "Power BI",
               "QlikView", "Looker", "Dataiku", "Alteryx", "Talend", "Informatica", "Matillion", "Databricks", "Snowflake",
               "Amazon Redshift", "Google BigQuery", "Microsoft Azure SQL Database", "Oracle Database", "MySQL", "PostgreSQL",
               "MongoDB", "Couchbase", "Cassandra", "Neo4j", "Apache Airflow", "Luigi", "Celery", "Apache NiFi", 
               "Talend Data Integration", "Google Cloud Dataflow", "Apache Beam", "Apache Flink", 
               "Microsoft Azure Data Factory", "IBM InfoSphere DataStage", "Apache Samza", "Apache Flume",
               "Apache Sqoop", "Microsoft Azure Databricks", "Teradata", "IBM Db2", "Apache Zeppelin", 
               "Jupyter Notebook", "Google Colab", "Anaconda", "TensorFlow", "PyTorch", "Keras",
               "scikit-learn", "XGBoost", "LightGBM", "H2O.ai", "Apache Mahout", "Microsoft Azure Machine Learning", 
               "IBM Watson", "RapidMiner", "KNIME", "DataRobot", "Amazon SageMaker", "Google Cloud AI Platform",
               "Domo", "IBM Cognos", "SAS", "Oracle Analytics Cloud", "Microsoft Power Automate",
               "Apache NiFi Registry", "Apache Atlas", "Apache Metron", "Apache Knox", "Apache Ranger",
               "Apache Superset", "Microsoft Power Apps", "Salesforce Einstein Analytics",
               "Google Data Studio", "Pentaho", "Microsoft Azure Synapse Analytics",
               "SAP Analytics Cloud", "MicroStrategy", "RStudio", "Apache Kylin", 
               "Apache HBase", "Apache Pig", "Apache Hive", "Apache Drill",
               "Cloudera Data Platform", "Hortonworks Data Platform", "Qubole", "DataRobot", "DataRobot Insights",
               "DataRobot MLOps", "DataRobot Paxata", "DataRobot AutoML", "DataRobot Time Series", "DataRobot AI Catalog"]

In [13]:
def create_tools(df, liste):
    tools_list = [tool.lower() for tool in liste]
    
    def find_tools(description):
        # Convertir la description en minuscules
        description = description.lower()

        # Créer une liste pour stocker les outils trouvés
        tools_found = []

        # Chercher chaque outil dans la description
        for tool in tools_list:
            if tool in description:
                tools_found.append(tool)
        
        # Si aucun outil n'a été trouvé, retourner NaN
        if not tools_found:
            return math.nan

        return ', '.join(tools_found)  # Retourner la liste d'outils trouvés
    
    df['tools'] = df['description'].apply(find_tools)
    return df

In [14]:
def new_skills(df):
    
    def clean_skills(list_skills_str):
        
        # Convertir la chaîne en une liste
        try:
            list_skills = ast.literal_eval(list_skills_str)
        except ValueError:
            # Si la conversion échoue, retourner la valeur telle quelle
            return list_skills_str
        
        # Assurez-vous que list_skills est une liste
        if isinstance(list_skills, list):
            clean_list = [skill.lower() for skill in list_skills]  # Convertir chaque compétence en minuscules
        else:
            # Si ce n'est pas une liste, retournez la valeur telle quelle
            return list_skills_str
            
        return ', '.join(clean_list)
    
    df['skills'] = df['skills'].apply(clean_skills)
    
    return df

In [15]:
df = create_tools(df, tools_list)

In [16]:
df = new_skills(df)

In [17]:
df.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools
0,"RESPONSABL SUPPLYCHAIN, LOGISTIQUE, DATA (H/F)",Le Groupe BIOTECH DENTAL renforce ses équipes ...,74 - SCIONZIER,Actualisé le 18 août 2023,39H Travail en journée,42000.0,Contrat à durée indéterminée,4 ans,"Bac+3, Bac+4 ou équivalents - Gestion producti...",[Analyser les coûts de la chaîne logistique (s...,Cadre,Décolletage,BIOTECH DENTAL MANUFACTURING,50 à 99 salariés,Fabricant de composants et sous ensembles méca...,NaN
1,Biostatisticien/Data-Manager (H/F),Avec plus de 20 ans d'expérience dans le dével...,75 - PARIS 15,Actualisé le 18 août 2023,39H Travail en journée,28000.0 - 30000.0,Contrat à durée déterminée - 12 Mois,Débutant accepté,NaN,"[Analyse statistique, Analyser, exploiter, str...",Agent de maîtrise,Recherche-développement en autres sciences phy...,Association ARTIC,6 à 9 salariés,Avec plus de 20 ans d expérience dans le dével...,sas
2,Data Analyst (H/F),Vous intégrez notre équipe Data pour interveni...,35 - RENNES,Actualisé le 18 août 2023,37H Travail en journée,35000.0 - 37000.0,Contrat à durée indéterminée,2 ans,Bac+5 et plus ou équivalents informatique déci...,"[Analyse statistique, Analyser, exploiter, str...",Cadre,Conseil en systèmes et logiciels informatiques,INETUM,0 salarié (n'ayant pas d'effectif au 31/12 mai...,NaN,"jira, tableau, power bi, looker, talend"
3,DATA ANALYST POUR CONCEPTION ESSAIS SOL A350 (...,"Valérie & Claire, vos Commandantes de Bord pou...",31 - BLAGNAC,Actualisé le 17 août 2023,35H Travail en journée,30000.0 - 35000.0,Mission intérimaire - 6 Mois,2 ans,NaN,"[Application web, Concevoir une application we...",Cadre,Activités des agences de travail temporaire,CRIT INTERIM,0 salarié (n'ayant pas d'effectif au 31/12 mai...,Acteur leader dans les ressources humaines et ...,NaN
4,Data scientist,Description de l'entreprise : \nExcellence Log...,92 - COLOMBES,Actualisé le 18 août 2023,39H Travail en journée,28000.0,Contrat à durée déterminée - 7 Mois,Débutant accepté,NaN,[Adapter les outils de traitement statistique ...,Cadre,"Ingénierie, études techniques",EXCELLENCE LOGGING FRANCE,10 à 19 salariés,Excellence Logging est une société de services...,pytorch


In [18]:
def transform_data(df):
    df = df.copy()
    
    df[['ID_dep', 'ville']] = df['loc'].str.split(' - ', expand = True)
    df['date'] = df['datePosted'].str.replace('Actualisé le ', '').str.replace('Publié le ', '')
    df['date'] = df['date'].apply(lambda x: dateparser.parse(x, languages=['fr']))
    df['date'] = df['date'].dt.strftime('%Y/%m/%d')
    
    df["jobs"] = df["jobs"].str.replace(" h/f", "")
    df["jobs"] = df["jobs"].str.replace("\s*\([hH]/[fF]\)\s*", " ").str.strip()
    df["jobs"] = df["jobs"].str.replace(" (H/F)", "")
    df["jobs"] = df["jobs"].str.replace(" H/F", "")
    df["jobs"] = df["jobs"].str.replace(" F/H", "")
    df["jobs"] = df["jobs"].str.replace(" (F/H)", "")
    df["jobs"] = df["jobs"].str.replace(" \(\)", "")

    
    df = df.applymap(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
    df = df.drop(["loc", "datePosted"], axis=1)

    return df

In [19]:
df = transform_data(df)

In [20]:
df.head()

,jobs,description,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,"RESPONSABL SUPPLYCHAIN, LOGISTIQUE, DATA",Le Groupe BIOTECH DENTAL renforce ses équipes ...,39H Travail en journée,42000.0,Contrat à durée indéterminée,4 ans,"Bac+3, Bac+4 ou équivalents - Gestion producti...",Analyser les coûts de la chaîne logistique (su...,Cadre,Décolletage,BIOTECH DENTAL MANUFACTURING,50 à 99 salariés,Fabricant de composants et sous ensembles méca...,NaN,74,SCIONZIER,2023/08/18
1,Biostatisticien/Data-Manager,Avec plus de 20 ans d'expérience dans le dével...,39H Travail en journée,28000.0 - 30000.0,Contrat à durée déterminée - 12 Mois,Débutant accepté,NaN,"Analyse statistique,Analyser, exploiter, struc...",Agent de maîtrise,Recherche-développement en autres sciences phy...,Association ARTIC,6 à 9 salariés,Avec plus de 20 ans d expérience dans le dével...,sas,75,PARIS 15,2023/08/18
2,Data Analyst,Vous intégrez notre équipe Data pour interveni...,37H Travail en journée,35000.0 - 37000.0,Contrat à durée indéterminée,2 ans,Bac+5 et plus ou équivalents informatique déci...,"Analyse statistique,Analyser, exploiter, struc...",Cadre,Conseil en systèmes et logiciels informatiques,INETUM,0 salarié (n'ayant pas d'effectif au 31/12 mai...,NaN,"jira, tableau, power bi, looker, talend",35,RENNES,2023/08/18
3,DATA ANALYST POUR CONCEPTION ESSAIS SOL A350 -...,"Valérie & Claire, vos Commandantes de Bord pou...",35H Travail en journée,30000.0 - 35000.0,Mission intérimaire - 6 Mois,2 ans,NaN,"Application web,Concevoir une application web,...",Cadre,Activités des agences de travail temporaire,CRIT INTERIM,0 salarié (n'ayant pas d'effectif au 31/12 mai...,Acteur leader dans les ressources humaines et ...,NaN,31,BLAGNAC,2023/08/17
4,Data scientist,Description de l'entreprise : \nExcellence Log...,39H Travail en journée,28000.0,Contrat à durée déterminée - 7 Mois,Débutant accepté,NaN,Adapter les outils de traitement statistique d...,Cadre,"Ingénierie, études techniques",EXCELLENCE LOGGING FRANCE,10 à 19 salariés,Excellence Logging est une société de services...,pytorch,92,COLOMBES,2023/08/18


In [21]:
def count_nan(df):
    
    nan_counts = df.isna().sum() # compte le nombre de NaN pour chaque colonne
    total_counts = len(df) # compte le nombre total de données dans le dataframe
    nan_percentages = (nan_counts / total_counts) * 100 # calcule le pourcentage de NaN pour chaque colonne
    result_df = pd.concat([nan_counts, nan_percentages], axis=1) # combine les deux séries en un dataframe
    result_df.columns = ['NaN Count', 'NaN Percentage'] # renomme les colonnes du nouveau dataframe
    result_df = result_df.sort_values(by = ['NaN Count'], ascending = False)
    return result_df

In [22]:
count_nan(df)

,NaN Count,NaN Percentage
diploma,652,98.787879
salary,575,87.121212
workhours,509,77.121212
qualifications,365,55.303030
tools,355,53.787879
industry,346,52.424242
company,292,44.242424
company_description,220,33.333333
size,182,27.575758
ville,53,8.030303


### Export Transform df 

In [23]:
def export_silver_to_csv(df):
    # Obtenez la date actuelle
    current_date = date.today().strftime("%d_%m_%Y")

    # Assurez-vous que le dossier de destination existe
    folder_path = f"../data/silver/{current_date}"
    os.makedirs(folder_path, exist_ok=True)

    # Exportez le DataFrame dans un fichier CSV
    csv_file_path = f"{folder_path}/jobs_data.csv"
    df.to_csv(csv_file_path, index=False)

In [24]:
export_silver_to_csv(df)

### Concat silver and transform df

In [25]:
def concat_data(df):
    
    # importer le df qui contient toutes les données déjà récoltées
    df_history = pd.read_csv("../data/silver/silver_data.csv")
    
    # concaténer les données qui viennent d'être récoltées à df_history
    global_df = pd.concat([df_history, df], axis=0)
    sort_df = global_df.sort_values('date', ascending=True)
    
    return sort_df

In [26]:
global_df = concat_data(df)

In [27]:
global_df.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"['Concevoir et gérer un projet', ""Concevoir un...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"[""Analyser les résultats d'un projet"", ""Déclin...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,['Contrôler et faire appliquer le respect de d...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13
3,Ingénieur Data H/F,"Vos missions\nAu sein de projets Agile Scrum, ...",66 - PERPIGNAN,Actualisé le 19 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,5 ans,NaN,"['Concevoir et gérer un projet', ""Concevoir un...",Cadre,"Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,100 à 199 salariés,"Azura, est un groupe familial, en forte croiss...","tableau, sas",66,PERPIGNAN,2023/07/19
4,Data Engineer pour la Qualité Totale H/F (H/F),Vous êtes Rattaché(e) au Responsable de l'équi...,92 - ST CLOUD,Actualisé le 19 juillet 2023,35H Travail en journée,38000.0 - 70000.0,Contrat à durée indéterminée,5 ans,NaN,"['Application web', 'Déterminer des mesures co...",Cadre,Activités des sièges sociaux,DASSAULT AVIATION,Non employeur (pas de salarié au cours de l'an...,Dassault Aviation,NaN,92,ST CLOUD,2023/07/19


In [28]:
global_df.shape

(2561, 19)

In [29]:
global_df.isnull().sum()

jobs                      0
description               0
loc                    1562
datePosted             1562
workhours              2051
salary                 2166
contract_type             0
experience                0
diploma                2479
skills                  861
qualifications         1437
industry               1377
company                1257
size                    888
company_description    1167
tools                  1285
ID_dep                    0
ville                   281
date                      0
dtype: int64

In [30]:
loc = global_df.loc[global_df.duplicated()]
loc.shape

(176, 19)

In [31]:
global_df = global_df.drop_duplicates()
global_df.duplicated().sum()

0

In [32]:
global_df["date"].min()

'2023/07/12'

In [33]:
global_df["date"].max()

'2023/08/18'

### Export silver df

In [34]:
def export_silver_df(df):
    csv_file_path = f"../data/silver/silver_data.csv"
    df.to_csv(csv_file_path, index=False)

In [35]:
export_silver_df(global_df)

In [36]:
df_final = pd.read_csv("../data/silver/silver_data.csv")
df_final.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"['Concevoir et gérer un projet', ""Concevoir un...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"[""Analyser les résultats d'un projet"", ""Déclin...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,['Contrôler et faire appliquer le respect de d...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13
3,Ingénieur Data H/F,"Vos missions\nAu sein de projets Agile Scrum, ...",66 - PERPIGNAN,Actualisé le 19 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,5 ans,NaN,"['Concevoir et gérer un projet', ""Concevoir un...",Cadre,"Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,100 à 199 salariés,"Azura, est un groupe familial, en forte croiss...","tableau, sas",66,PERPIGNAN,2023/07/19
4,Data Engineer pour la Qualité Totale H/F (H/F),Vous êtes Rattaché(e) au Responsable de l'équi...,92 - ST CLOUD,Actualisé le 19 juillet 2023,35H Travail en journée,38000.0 - 70000.0,Contrat à durée indéterminée,5 ans,NaN,"['Application web', 'Déterminer des mesures co...",Cadre,Activités des sièges sociaux,DASSAULT AVIATION,Non employeur (pas de salarié au cours de l'an...,Dassault Aviation,NaN,92,ST CLOUD,2023/07/19


In [37]:
df.shape

(660, 17)

In [38]:
def count_nan(df):
    nan_counts = df.isna().sum() # compte le nombre de NaN pour chaque colonne
    value_counts = df.count() # compte le nombre de valeurs non-NaN pour chaque colonne
    total_counts = len(df) # compte le nombre total de données dans le dataframe
    nan_percentages = (nan_counts / total_counts) * 100 # calcule le pourcentage de NaN pour chaque colonne
    result_df = pd.concat([value_counts, nan_counts, nan_percentages], axis=1) # combine les séries en un dataframe
    result_df.columns = ['True Values', 'NaN Count', 'NaN Percentage'] # renomme les colonnes du nouveau dataframe
    return result_df

In [39]:
NaN = count_nan(df_final)
NaN = NaN.sort_values(by = ['NaN Count'], ascending = False)
NaN

,True Values,NaN Count,NaN Percentage
diploma,82,2303,96.561845
salary,374,2011,84.318658
workhours,508,1877,78.700210
loc,999,1386,58.113208
datePosted,999,1386,58.113208
skills,1064,1321,55.387841
qualifications,1080,1305,54.716981
industry,1133,1252,52.494759
tools,1180,1205,50.524109
company,1265,1120,46.960168
